# Keyword Search

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
# !pip install cohere
# !pip install weaviate-client

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Let's start by imporing Weaviate to access the Wikipedia database.

In [19]:
import weaviate
auth_config = weaviate.auth.AuthApiKey(api_key = "76320a90-53d8-42bc-b41d-678647c6672e")

In [24]:
client = weaviate.Client(url = "https://cohere-demo.weaviate.network/",
                         auth_client_secret = auth_config,
                         additional_headers = {"X-Cohere-Api-Key":
                                               "c70AvJ6DUm1S6kwQJUyJsPEAI683RH2DjOr7QPiG",
                                              })

In [25]:
client.is_ready() 

True

# Keyword Search

In [35]:
def keyword_search(query,
                   results_lang = "en",
                   properties = ["title", "url", "text"],
                   num_results= 5):

    where_filter = {"path": ["lang"],
                    "operator": "Equal",
                    "valueString": results_lang}
    
    response = (client.query.get("Articles", properties)
                .with_bm25(query=query)
                .with_where(where_filter)
                .with_limit(num_results)
                .do())
    
    result = response["data"]["Get"]["Articles"]
    return result

#El formato no es muy bueno

In [36]:
query = "Who was James Clerk Maxwell?"
keyword_search_results = keyword_search(query)
print(keyword_search_results)

[{'text': 'James Clerk Maxwell was born on 13 June 1831 at 14 India Street, Edinburgh, to John Clerk Maxwell of Middlebie, an advocate, and Frances Cay, daughter of Robert Hodshon Cay and sister of John Cay. (His birthplace now houses a museum operated by the James Clerk Maxwell Foundation.) His father was a man of comfortable means of the Clerk family of Penicuik, holders of the baronetcy of Clerk of Penicuik. His father\'s brother was the 6th baronet. He had been born "John Clerk", adding "Maxwell" to his own after he inherited (as an infant in 1793) the Middlebie estate, a Maxwell property in Dumfriesshire. James was a first cousin of both the artist Jemima Blackburn (the daughter of his father\'s sister) and the civil engineer William Dyce Cay (the son of his mother\'s brother). Cay and Maxwell were close friends and Cay acted as his best man when Maxwell married.', 'title': 'James Clerk Maxwell', 'url': 'https://en.wikipedia.org/wiki?curid=28989696'}, {'text': 'Maxwell is buried a

### Try modifying the search options
- Other languages to try: `en, de, fr, es, it, ja, ar, zh, ko, hi`

In [37]:
properties = ["text", "title", "url", "views", "lang"]

In [38]:
#Mejoramos el formato

def print_result(result):
    """ Print results with colorful formatting """
    for i, item in enumerate(result):
        print(f"item {i}")
        for key in item.keys():
            print(f"{key}: {item.get(key)}")
            print()
        print()

In [39]:
print_result(keyword_search_results)

item 0
text: James Clerk Maxwell was born on 13 June 1831 at 14 India Street, Edinburgh, to John Clerk Maxwell of Middlebie, an advocate, and Frances Cay, daughter of Robert Hodshon Cay and sister of John Cay. (His birthplace now houses a museum operated by the James Clerk Maxwell Foundation.) His father was a man of comfortable means of the Clerk family of Penicuik, holders of the baronetcy of Clerk of Penicuik. His father's brother was the 6th baronet. He had been born "John Clerk", adding "Maxwell" to his own after he inherited (as an infant in 1793) the Middlebie estate, a Maxwell property in Dumfriesshire. James was a first cousin of both the artist Jemima Blackburn (the daughter of his father's sister) and the civil engineer William Dyce Cay (the son of his mother's brother). Cay and Maxwell were close friends and Cay acted as his best man when Maxwell married.

title: James Clerk Maxwell

url: https://en.wikipedia.org/wiki?curid=28989696


item 1
text: Maxwell is buried at Parto

In [40]:
query = "¿Quién ha sido el mejor músico de la historia?"
keyword_search_results = keyword_search(query, results_lang = "es")
print_result(keyword_search_results)

item 0
text: Los Hawks de Dominique Wilkins eran un escollo muy duro de pelar. En 1988 en una de las grandes series de playoffs que se recuerdan, Wilkins, un asiduo al espectáculo, inmerso en su pura esencia, protagoniza, de la mano del mejor alero que ha parido la NBA, Larry Bird, la siguiente heroicidad traducida en forma de partido-duelo. Dicha historia trata de retroceder en el tiempo para llevarnos a uno de los mejores encuentros que se han podido visualizar en Playoffs (finales aparte): Boston Celtics y Atlanta Hawks marchaban con la serie empatada a 3, encuentro a vida o muerte para discernir quién acompañaría a Detroit en la Final de Conferencia Este.

title: Historia de la NBA

url: https://es.wikipedia.org/wiki?curid=1001161


item 1
text: Los historiadores someten a los evangelios a un análisis crítico, en un intento de diferenciar mejor lo auténtico, información fidedigna de posibles invenciones, exageraciones, y alteraciones. Puesto que hay más variantes textuales en el Nu

Los resultados son muy malos. Se necesita implementar algo para mejorar esto.